In [1]:
import numpy as np
import os
import tensorflow as tf

In [2]:
X_cal_data = np.load('./calibration data/X_cal_data.npy')
Y_cal_data = np.load('./calibration data/Y_cal_data.npy')

In [3]:
print(X_cal_data.shape)

(50, 150, 1152)


In [4]:
save_dir = 'cal_Data'
os.makedirs(save_dir, exist_ok=True)

for i in range(X_cal_data.shape[0]):
    filename = os.path.join(save_dir, f'cal_{i+1}.npy')
    reshaped_data = X_cal_data[i].reshape(150, 1152)
    # Create an empty file
    with open(filename, 'wb') as f:
        pass
    # Save the sub-array to the newly created file
    np.save(filename, reshaped_data)

print("Files saved successfully.")

Files saved successfully.


In [5]:
cal_1 = np.load('./cal_Data/cal_1.npy')

In [6]:
print(cal_1.shape)

(150, 1152)


In [7]:
X_cal_data[0]

array([[0.85882353, 0.30588235, 1.        , ..., 0.48627451, 0.55294118,
        0.69411765],
       [0.38431373, 0.5372549 , 1.        , ..., 1.        , 0.67843137,
        0.23529412],
       [0.25882353, 0.55686275, 1.        , ..., 0.75294118, 0.55294118,
        0.36078431],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [8]:
cal_1[0]

array([0.85882353, 0.30588235, 1.        , ..., 0.48627451, 0.55294118,
       0.69411765])

# Inferencing on Calibration data

In [11]:
# Load the TFLite model and allocate tensors
interpreter = tf.lite.Interpreter(model_path='./cnn_model_new.tflite')
# interpreter.allocate_tensors()
# # Get input and output tensors
# input_details = interpreter.get_input_details()/
# output_details = interpreter.get_output_details()

In [21]:
interpreter.allocate_tensors()

In [13]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [14]:
def run_inference(input_data):
    """
    Run inference on a single input using the TensorFlow Lite interpreter.

    Parameters:
    input_data (np.ndarray): The input data for the model. Should be a NumPy array with the appropriate shape.

    Returns:
    output_data (np.ndarray): The output data from the model. The shape and type of this output depend on the model's architecture.
    """
    interpreter.set_tensor(input_details[0]['index'], input_data)
    interpreter.invoke()
    output_data = interpreter.get_tensor(output_details[0]['index'])
    return output_data

In [15]:
def evaluate_model(interpreter, X_test, y_test, threshold=0.5):
    """
    Evaluate the TensorFlow Lite model on the given test dataset.

    Parameters:
    interpreter (tf.lite.Interpreter): The TensorFlow Lite interpreter for running inference.
    X_test (np.ndarray): The test dataset input features. Should be a NumPy array of shape (n_samples, seq_len, n_features).
    y_test (np.ndarray): The test dataset labels. Should be a NumPy array of shape (n_samples,).
    threshold (float, optional): he threshold for binary classification. Default is 0.5.

    Returns:
    accuracy (float): The accuracy of the model on the test dataset.
    """
    correct_predictions = 0
    total_predictions = 0

    for i in range(len(X_test)):
        input_data = np.expand_dims(X_test[i], axis=0).astype(np.float32)  # Adding batch dimension
        predicted_output = run_inference(input_data)
        predicted_label = (predicted_output >= threshold).astype(int)[0][0]  # Applying threshold
        true_label = y_test[i]

        if predicted_label == true_label:
            correct_predictions += 1
        total_predictions += 1

    accuracy = correct_predictions / total_predictions
    return accuracy

In [16]:
accuracy = evaluate_model(interpreter, X_cal_data, Y_cal_data)
print(f'Accuracy: {accuracy}')

Accuracy: 0.96


In [18]:
import tensorflow as tf
import numpy as np

# Load the TFLite model
model_path = 'cnn_model_new.tflite'
interpreter = tf.lite.Interpreter(model_path=model_path)
interpreter.allocate_tensors()

# Verify input and output tensor details
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

print("Input details:")
for detail in input_details:
    print(f"Name: {detail['name']}")
    print(f"Index: {detail['index']}")
    print(f"Shape: {detail['shape']}")
    print(f"Shape Signature: {detail.get('shape_signature', 'N/A')}")
    print(f"Dtype: {detail['dtype']}")
    print(f"Quantization: {detail['quantization']}")
    print()

print("Output details:")
for detail in output_details:
    print(f"Name: {detail['name']}")
    print(f"Index: {detail['index']}")
    print(f"Shape: {detail['shape']}")
    print(f"Shape Signature: {detail.get('shape_signature', 'N/A')}")
    print(f"Dtype: {detail['dtype']}")
    print(f"Quantization: {detail['quantization']}")
    print()


Input details:
Name: serving_default_reshape_input:0
Index: 0
Shape: [   1  150 1152]
Shape Signature: [   1  150 1152]
Dtype: <class 'numpy.float32'>
Quantization: (0.0, 0)

Output details:
Name: StatefulPartitionedCall:0
Index: 26
Shape: [1 1]
Shape Signature: [1 1]
Dtype: <class 'numpy.float32'>
Quantization: (0.0, 0)



In [19]:
cal_image_1 = np.load('./cal_Data/cal_1.npy')

In [20]:
interpreter = tf.lite.Interpreter(model_path='./cnn_model_new.tflite')

In [23]:
interpreter.allocate_tensors()

In [24]:
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

In [25]:
floating_model = input_details[0]['dtype'] == np.float32

In [26]:
print(floating_model)

True


In [29]:
batch = input_details[0]['shape'][0]
print(batch)

1


In [31]:
height = input_details[0]['shape'][1]
print(height)

150


In [32]:
width = input_details[0]['shape'][2]
print(width)

1152
